In [1]:
pip install gdown

In [2]:
import gdown

gdown.download("https://drive.google.com/uc?id=1f8eaAZY-7YgFxLcrL3OkvSRa3onNNLb9")

Downloading...
From (original): https://drive.google.com/uc?id=1f8eaAZY-7YgFxLcrL3OkvSRa3onNNLb9
From (redirected): https://drive.google.com/uc?id=1f8eaAZY-7YgFxLcrL3OkvSRa3onNNLb9&confirm=t&uuid=9b8d3f4d-3a20-4e96-926b-a011a926426d
To: /content/midterm-regresi-dataset.csv
100%|██████████| 443M/443M [00:03<00:00, 141MB/s]


'midterm-regresi-dataset.csv'

In [3]:
import pandas as pd


# Dataset ini tidak punya header.
# Nilai seperti 2001 di kolom pertama adalah TARGET (tahun), bukan nama feature /  kolom.
file_path = '/content/midterm-regresi-dataset.csv'
df = pd.read_csv(file_path)

df.head()

,2001,49.94357,21.47114,73.0775,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.0162,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
0,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
1,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
2,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
3,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
4,2001,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


In [4]:
print("df.shape:", df.shape) #ada 515344 row/baris dan 90 feature

df.shape: (515344, 91)


In [5]:
import os
file_size_bytes = os.path.getsize(file_path)
file_size_mb = file_size_bytes / (1024 ** 2)

print(f"Ukuran file: {file_size_bytes:,} bytes")
print(f"Ukuran file: {file_size_mb:.2f} MB")  #ukuran 422.88 MB, deadline 2 minggu. ngerjain mepet pasti ga kelar :))))

Ukuran file: 443,423,087 bytes
Ukuran file: 422.88 MB


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("🚀 1. FIX LOAD DATA & PREPROCESSING (DL)...")

file_path = '/content/midterm-regresi-dataset.csv'

# PERBAIKAN: Tambahkan header=None
df = pd.read_csv(file_path, header=None)

print(f"✅ Data Loaded Benar. Shape: {df.shape}")
# Harusnya (515345, 91) -> Barisnya ganjil karena baris 1 tidak hilang

# 2. Pisahkan Target (Tahun) dan Fitur (Audio)
# Kolom 0 = Tahun, Kolom 1-90 = Fitur
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# 3. Scaling (WAJIB untuk Deep Learning)
# Neural Network butuh angka skala kecil (-1 s/d 1) agar bisa belajar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Split Train & Test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"✅ Data Siap DL. Train: {X_train.shape}, Test: {X_test.shape}")

🚀 1. FIX LOAD DATA & PREPROCESSING (DL)...
✅ Data Loaded Benar. Shape: (515345, 91)
✅ Data Siap DL. Train: (412276, 90), Test: (103069, 90)


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

print("🧠 LANGKAH 2: Membangun Arsitektur Model...")

# 1. Definisikan Arsitektur Neural Network
# Kita pakai arsitektur yang cukup dalam (Deep) untuk pola rumit pada audio
model = models.Sequential([
    # Input Layer (90 Fitur)
    layers.Input(shape=(X_train.shape[1],)),

    # Hidden Layer 1
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.2),

    # Hidden Layer 2
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),

    # Hidden Layer 3
    layers.Dense(64, activation='relu'),

    # Output Layer: 1 Neuron (Tahun), Linear/Tanpa Aktivasi
    layers.Dense(1)
])

# 2. Compile Model
# Loss: MSE (Mean Squared Error) - Wajib untuk Regresi
# Hyperparameter Tuning (Basic Level): Set Learning Rate spesifik
optimizer = optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
              loss='mse',
              metrics=['mae']) # MAE agar error gampang dibaca

model.summary()

🧠 LANGKAH 2: Membangun Arsitektur Model...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        23,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,513 (252.00 KB)

 Trainable params: 64,513 (252.00 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping

print("🔥 LANGKAH 3: Training Deep Learning...")

# Early Stopping: Berhenti kalau model tidak bertambah pintar setelah 10 epoch
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stop], # Terapkan early stopping
    verbose=1
)

# Evaluasi Akhir
loss, mae = model.evaluate(X_test, y_test, verbose=0)

print("\n📊 HASIL EVALUASI DEEP LEARNING:")
print(f"MAE (Rata-rata Meleset): {mae:.2f} Tahun")
print("🎉 TUGAS DEEP LEARNING REGRESI SELESAI!")

🔥 LANGKAH 3: Training Deep Learning...
Epoch 1/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 560346.6875 - mae: 473.5381 - val_loss: 4919.5151 - val_mae: 39.5491
Epoch 2/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 18859.4512 - mae: 107.5694 - val_loss: 1229.2211 - val_mae: 23.0204
Epoch 3/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 14808.3652 - mae: 96.7647 - val_loss: 782.5126 - val_mae: 19.1646
Epoch 4/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 13757.7363 - mae: 93.4109 - val_loss: 769.4308 - val_mae: 19.2239
Epoch 5/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 12913.7783 - mae: 90.3700 - val_loss: 620.8793 - val_mae: 18.1750
Epoch 6/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 12132.7402 - mae: 87.6252 - val_loss: 569.0060 - val_mae: 16.6911
Epoch 7/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 11098.3037 - mae: 83.9423 - val_loss: 913.0901 - val_mae: 24.4255
Epoch 8/50
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - lo